In [2]:
using DataFrames, XLSX, DataFramesMeta
using LinearAlgebra
using LightGraphs, SimpleWeightedGraphs

┌ Info: Recompiling stale cache file C:\Users\cilli\.julia\compiled\v1.2\DataFramesMeta\2xRTO.ji for DataFramesMeta [1313f7d8-7da2-5740-9ea0-a2ca25f37964]
└ @ Base loading.jl:1240
┌ Info: Recompiling stale cache file C:\Users\cilli\.julia\compiled\v1.2\LightGraphs\Xm08G.ji for LightGraphs [093fc24a-ae57-5d10-9952-331d41423f4d]
└ @ Base loading.jl:1240
┌ Info: Recompiling stale cache file C:\Users\cilli\.julia\compiled\v1.2\SimpleWeightedGraphs\iIsRI.ji for SimpleWeightedGraphs [47aef6b3-ad0c-573a-a1e2-d07658019622]
└ @ Base loading.jl:1240


# Network Data

In [3]:
Bus = DataFrame(XLSX.readtable("All-Ireland System- WinterPeak 2026 - PSSE Data.xlsx", "BusData", infer_eltypes=true)... );

In [4]:
BusDist =  @linq Bus |>
            where(:kV .<= 38) |>
            select(:Bus,:kV);
size(BusDist)

(1565, 2)

In [5]:
BusTrans =  @linq Bus |>
            where(:kV .> 38) |>
            select(:Bus,:kV);
BusNo=size(BusTrans,1)

545

In [25]:
BranchFrame = DataFrame(XLSX.readtable("IrishSystemData.xlsx", "AC-Line", infer_eltypes=true)... )
show(BranchFrame,allcols = true)

1068×13 DataFrame
│ Row  │ From  │ From Bus  Name     │ To    │ To Bus  Name       │ Id    │
│      │ Int64 │ String             │ Int64 │ String             │ Int64 │
├──────┼───────┼────────────────────┼───────┼────────────────────┼───────┤
│ 1    │ 11    │ FMN LOOP    38.000 │ 42404 │ NORTH_WALL  38.000 │ 1     │
│ 2    │ 12    │ BEDFORD ROW 38.000 │ 25404 │ FRANCIS ST  38.000 │ 1     │
│ 3    │ 12    │ BEDFORD ROW 38.000 │ 25404 │ FRANCIS ST  38.000 │ 3     │
│ 4    │ 12    │ BEDFORD ROW 38.000 │ 46504 │ RINGSEND    38.000 │ 1     │
│ 5    │ 12    │ BEDFORD ROW 38.000 │ 46504 │ RINGSEND    38.000 │ 2     │
│ 6    │ 1021  │ ARDNACRUSHA 110.00 │ 2121  │ DRUMLINE    110.00 │ 1     │
│ 7    │ 1021  │ ARDNACRUSHA 110.00 │ 2361  │ ENNIS       110.00 │ 1     │
│ 8    │ 1021  │ ARDNACRUSHA 110.00 │ 3541  │ LIMERICK    110.00 │ 1     │
│ 9    │ 1021  │ ARDNACRUSHA 110.00 │ 10210 │ ARDNACRUSHA 110.00 │ 1     │
│ 10   │ 1021  │ ARDNACRUSHA 110.00 │ 10261 │ ARDNA_CAP   110.00 │ 1     │
⋮
│ 105

In [26]:
From1 = BranchFrame.From;
To1 = BranchFrame.To
R1 = BranchFrame.R;
X1 = BranchFrame.X;
B1 = BranchFrame.B;

In [27]:
BranchFrame = DataFrame(XLSX.readtable("IrishSystemData.xlsx", "AC-Line", infer_eltypes=true)... )
show(BranchFrame,allcols = true)

1068

In [32]:
filter(row -> BranchFrame ∈ [1,2,3], BranchFrame )

,From,From Bus Name,To,To Bus Name,Id,Name,Term Node Num (From),Term Node Name (From)
,Int64,String,Int64,String,Int64,Any,Any,Any


In [22]:
Gw = SimpleWeightedGraph(BusNo)

{545, 0} undirected simple Int64 graph with Float64 weights

In [64]:
BranchInter1 = Branch[findall(in(BusTrans.Bus), Branch.From),:];
BranchInter1

,From,From Bus Name,To,To Bus Name,Id,Name,Term Node Num (From)
,Int64,String,Int64,String,Int64,Any,Any
1,1021,ARDNACRUSHA 110.00,2121,DRUMLINE 110.00,1,missing,missing
2,1021,ARDNACRUSHA 110.00,2361,ENNIS 110.00,1,missing,missing
3,1021,ARDNACRUSHA 110.00,3541,LIMERICK 110.00,1,missing,missing
4,1021,ARDNACRUSHA 110.00,10210,ARDNACRUSHA 110.00,1,missing,missing
5,1021,ARDNACRUSHA 110.00,10261,ARDNA_CAP 110.00,1,missing,missing
6,1041,AGHADA 110.00,5481,WHITEGATE 110.00,1,missing,missing
7,1042,AGHADA 220.00,1052,AGHADA_B 220.00,1,missing,missing
8,1042,AGHADA 220.00,1072,AGHT2102 220.00,1,missing,missing
9,1042,AGHADA 220.00,3202,KNOCKRAHA 220.00,1,missing,missing


In [66]:
BranchInter2 = Branch[findall(in(BusTrans.Bus), Branch.From),:];
BranchInter2 

,From,From Bus Name,To,To Bus Name,Id,Name,Term Node Num (From)
,Int64,String,Int64,String,Int64,Any,Any
1,1021,ARDNACRUSHA 110.00,2121,DRUMLINE 110.00,1,missing,missing
2,1021,ARDNACRUSHA 110.00,2361,ENNIS 110.00,1,missing,missing
3,1021,ARDNACRUSHA 110.00,3541,LIMERICK 110.00,1,missing,missing
4,1021,ARDNACRUSHA 110.00,10210,ARDNACRUSHA 110.00,1,missing,missing
5,1021,ARDNACRUSHA 110.00,10261,ARDNA_CAP 110.00,1,missing,missing
6,1041,AGHADA 110.00,5481,WHITEGATE 110.00,1,missing,missing
7,1042,AGHADA 220.00,1052,AGHADA_B 220.00,1,missing,missing
8,1042,AGHADA 220.00,1072,AGHT2102 220.00,1,missing,missing
9,1042,AGHADA 220.00,3202,KNOCKRAHA 220.00,1,missing,missing


In [69]:
Comparison = BranchInter1.From .== BranchInter2.From;
unique(Comparison)

1-element Array{Bool,1}:
 1

### Define the bus set

In [32]:
Bus = max(length(unique(Branch.From)),length(unique(Branch.To)))

822

### Defining Susceptance Matrix (Laplacian)
##### Note: only Susceptance Matirx is defined for Linear power flow

In [33]:
Gw = SimpleWeightedGraph(BusNo)

for i = 1:size(Branch,1)
    add_edge!(Gw,Branch.From[i],Branch.To[i], 1/Branch.R[i])
end

In [34]:
Aw = Array(weights(Gw))

822×822 Array{Float64,2}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.

In [ ]:
Aw = Array(weights(Gw))
Lw = Diagonal(sum(Aw,dims=1)[:])-Aw
B = Lw;

# Generator Location Data

In [5]:
#GenData = DataFrame(a = [0.001562, 0.00194, 0.00482], 
#                b = [7.92, 7.85, 7.97],
#                c = [561, 310, 78],
#                Pmax = [600, 400, 200],
#                Pmin = [150, 100, 50],
#                RampUp = [30,20,30],
#                RampDown = [30,20,30],
#                MinUp = [2,3,2],
#                MinDown = [2,3,2]);

In [6]:
GenData = DataFrame(XLSX.readtable("3-Bus.xlsx", "GenData", infer_eltypes=true)... )

,Name,Type,a,b,c,Pmax,Pmin,RampUp,RampDown,MinUp
,Int64,String,Float64,Float64,Int64,Int64,Int64,Int64,Int64,Int64
1,1,Unknown,0.001562,7.92,561,600,150,30,30,2
2,2,Unknown,0.00194,7.85,310,400,100,20,20,3
3,3,Unknown,0.00482,7.97,78,200,50,30,30,2


In [7]:
PVar = DataFrame(Bus = [1, 2, 3], Gen = [1,1,1])

Bus = max(maximum(Branch.From),maximum(Branch.To))
Genw = SimpleDiGraph(Bus)

for i = 1:size(PVar,1)
    add_edge!(Genw,PVar.Bus[i],PVar.Gen[i] )
    println(PVar.Bus[i],PVar.Gen[i])
end
AGenw = Array(adjacency_matrix(Genw))

11
21
31


3×3 Array{Int64,2}:
 1  0  0
 1  0  0
 1  0  0

### Define Generator Set

In [8]:
Gen = size(GenData, 1)

3